- base: https://www.kaggle.com/code/vadimkamaev/postprocessin-ensemble
- select feats by importances and 3 ensembles (no postprocess) 
- focus on tree modelling

# common class

In [1]:
import sys
import optuna
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold, KFold

class TreeModelling:
    def __init__(self, config):
        self.config = config 
        
    def lgb_train_and_pred(self, x_tr, y_tr, x_val, y_val, test, feats, split):
    
        feat_imp_df = pd.DataFrame(feats, columns = ["feat"])
        model = lgb.train(self.config["lgb"]["params"], lgb.Dataset(x_tr, y_tr), self.config["lgb"]["num_round"], valid_sets=[lgb.Dataset(x_val, y_val)], 
                      callbacks = [lgb.early_stopping(stopping_rounds=self.config["lgb"]["es_round"], verbose=True), 
                                   lgb.log_evaluation(self.config["lgb"]["verbose_eval"])]) 
        feat_imp_df["imp"+str(split)] = model.feature_importance()
        valid_pred = model.predict(x_val)
        test_pred = model.predict(test) 
    
        return test_pred, valid_pred, feat_imp_df
    

    def xgb_train_and_pred(self, x_tr, y_tr, x_val, y_val, test, feats, split):

        xgb_eval = xgb.DMatrix(x_val, label = y_val)
        model = xgb.train(self.config["xgb"]["params"], xgb.DMatrix(x_tr, label = y_tr), self.config["xgb"]["num_round"], evals = [(xgb_eval, "eval")], 
                          early_stopping_rounds=self.config["xgb"]["es_round"], verbose_eval = self.config["xgb"]["verbose_eval"])
        feat_imp_df = pd.DataFrame(model.get_score(importance_type="total_gain").items(), columns =["feat", "imp"+str(split)])
        valid_pred = model.predict(xgb_eval, iteration_range=(0, model.best_ntree_limit))
        dtest = xgb.DMatrix(test)
        test_pred = model.predict(dtest, iteration_range=(0, model.best_ntree_limit))
    
        return test_pred, valid_pred, feat_imp_df

    
    def cb_train_and_pred(self, x_tr, y_tr, x_val, y_val, test, feats, split):
        
        feat_imp_df = pd.DataFrame(feats, columns = ["feat"]) 
        if self.config["task_type"] == "classification":
            model = CatBoostClassifier(**self.config["cb"]["params"])
            model.fit(x_tr, y_tr, eval_set=[(x_val, y_val)], early_stopping_rounds=self.config["cb"]["es_round"],
                      verbose_eval = self.config["cb"]["verbose_eval"])
            valid_pred = model.predict_proba(x_val)[:,1]
            test_pred = model.predict_proba(test)[:,1]
        elif self.config["task_type"]:
            model = CatBoostRegressor(**self.config["cb"]["params"])
            model.fit(x_tr, y_tr, eval_set=[(x_val, y_val)], early_stopping_rounds=self.config["cb"]["es_round"], 
                      verbose_eval = self.config["cb"]["verbose_eval"])
            valid_pred = model.predict(x_val)
            test_pred = model.predict(test)
        
        feat_imp_df["imp"+str(split)] = model.get_feature_importance()
    
        return test_pred, valid_pred, feat_imp_df
        
    
    def cv_execute(self, model_name):
            
        feats = self.config[model_name]["feats"]
        valid_output = np.zeros(len(self.config["target"]))
        cv = self.config["cv"]
        if model_name == "lgb":
            func = self.lgb_train_and_pred
        elif model_name == "xgb":
            func = self.xgb_train_and_pred
        elif model_name == "cb":
            func = self.cb_train_and_pred
    
        for split, (tr_idx, val_idx) in enumerate(cv.split(self.config["train_data"], self.config["fold_y"])): 
            x_tr = self.config["train_data"][feats].iloc[tr_idx]
            x_val = self.config["train_data"][feats].iloc[val_idx]
            y_tr = self.config["target"].iloc[tr_idx]
            y_val = self.config["target"].iloc[val_idx]

            if split == 0:
                test_output, valid_pred, feat_imp_df = func(x_tr, y_tr, x_val, y_val, self.config["test_data"][feats], feats, split)
            else:
                tmp_output, valid_pred, tmp_imp_df = func(x_tr, y_tr, x_val, y_val, self.config["test_data"][feats], feats, split)
                test_output += tmp_output
                feat_imp_df = pd.merge(feat_imp_df, tmp_imp_df, on = "feat", how = "inner")
            valid_output[val_idx] = valid_pred
    
        test_output /= (split + 1)
        feat_imp_df["sum"] = feat_imp_df.iloc[:,1:].sum(axis=1)
        
        val_score = self.config["metric_function"](self.config["target"], valid_output)
    
        return test_output, valid_output, feat_imp_df
    

    def lgb_objective(self, trial):
        # https://lightgbm.readthedocs.io/en/latest/Parameters.html
        check_params = {
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 10),
            'lambda_l2': trial.suggest_float('lambda_l2', 0, 0.1),
            'lambda_l1': trial.suggest_float('lambda_l1', 0, 0.1),
        }
    
        params = self.config["lgb"]["params"].copy()
        params.update(check_params)
        _, val_output, _ = self.cv_execute("lgb")
        score = self.config["metric_function"](self.config["target"], val_output)
        return score

    
    def xgb_objective(self, trial):
        # https://xgboost.readthedocs.io/en/stable/parameter.html
        check_params = {
            'gamma': trial.suggest_float('gamma', 0, 1),
            'max_depth': trial.suggest_int('max_depth', 1, 10),
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
            'subsample': trial.suggest_float('subsample', 0, 1),
            'lambda': trial.suggest_float('lambda', 0, 1),
            'alpha': trial.suggest_float('alpha', 0, 1),
        }
    
        params = self.config["xgb"]["params"].copy()
        params.update(check_params)
        _, val_output, _ = self.cv_execute("xgb")
        score = self.config["metric_function"](self.config["target"], val_output)
        return score

    
    def cb_objective(self, trial):
        # https://catboost.ai/en/docs/references/training-parameters/
        check_params = {
            'max_depth': trial.suggest_int('max_depth', 1, 10),
            #'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
            'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.5),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 10),
            'colsample_bylevel': trial.suggest_float('reg_lambda', 0, 1.0),
        }
    
        params = self.config["cb"]["params"].copy()
        params.update(check_params)
        _, val_output, _ = self.cv_execute("cb")
        score = self.config["metric_function"](self.config["target"], val_output) 
        return score

    
    def param_tuning(self, model_name, option = "minimize"):

        if model_name == "lgb":
            objective = self.lgb_objective
        elif model_name == "xgb":
            objective = self.xgb_objective
        elif model_name == "cb":
            objective = self.cb_objective
            
        study = optuna.create_study(direction=option) 
        study.optimize(objective, n_trials=self.config["optuna_trial_num"])
        trial = study.best_trial
        print('Value: ', trial.value)
        return trial.params

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# preprocess

In [2]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')


import itertools
from datetime import datetime
from sklearn.decomposition import PCA

train['EJ'] = train['EJ'].map({'A': 0, 'B': 1})
test['EJ']  = test['EJ'].map({'A': 0, 'B': 1})

# process epsilon
train = pd.merge(train, greeks, on = "Id", how = "inner")
train_stratify = train[["Class", "Beta", "Delta", "Gamma"]] 
train["Epsilon_ordinal"] = train["Epsilon"].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal() if x != "Unknown" else np.nan)

org_features = [n for n in train.columns if n not in ['Class', 'Id', 'Alpha', "Beta", "Gamma", "Delta", "Epsilon"]]
test_times = pd.DataFrame([train.Epsilon_ordinal.max() + 1] * len(test), columns = ["Epsilon_ordinal"])
final_test = pd.concat((test, test_times), axis=1)

# fill missing value
train.fillna(-999, inplace=True)
final_test.fillna(-999, inplace=True)

# add pca columns
pca_feat_num = 15
pca_cols = ["pca"+str(i+1) for i in range(pca_feat_num)]
pca = PCA(n_components=pca_feat_num,random_state=42)
pca_train = pca.fit_transform(train[org_features])
pca_test = pca.transform(final_test[org_features])
pca_train = pd.DataFrame(pca_train, columns=pca_cols)
pca_test = pd.DataFrame(pca_test, columns=pca_cols)
train = pd.concat([train, pca_train],axis=1)
final_test = pd.concat([final_test, pca_test],axis=1)

print(train.shape, final_test.shape)

(617, 79) (5, 73)


# config

In [3]:
from sklearn.metrics import log_loss
def balanced_log_loss(y_true, y_pred):
    nc = np.bincount(y_true)
    return log_loss(y_true, y_pred, sample_weight = 1/nc[y_true], eps=1e-15)

config = {
    "xgb": {
        "feats": org_features + pca_cols,
        "num_round" : 1000,
        "es_round" : 50,
        "verbose_eval": 100,
        "params" : {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'verbosity': 0,
            'seed': 42},   
    },
    
    "lgb": {
        "feats": org_features + pca_cols,
        "num_round" : 1000,
        "es_round" : 20,
        "verbose_eval": 100,
        "params" : {
            'objective': 'binary', 
            'metric': 'binary_logloss', 
            'boosting': 'goss',
            'verbose': -1,
            'seed': 42,
            'is_unbalance': True},   
    },

    "cb": {
        "feats": org_features + pca_cols,
        "es_round" : 40,
        "verbose_eval": 100,
        "params" : {
            'loss_function': 'Logloss',
            'iterations': 1000,  
            'random_seed': 42,
            'use_best_model' : True},
    },

    "task_type": "classification",
    "cv": MultilabelStratifiedKFold(5, shuffle=True, random_state=42),
    "train_data": train,
    "fold_y": train_stratify,
    "test_data": final_test,
    "target": train.Class,
    "metric_function" : balanced_log_loss,
    "optuna_trial_num": 5,
}

# tree model

In [4]:
treemodel = TreeModelling(config)

In [5]:
_, lgb_val_output, lgb_imp_df = treemodel.cv_execute("lgb")
lgb_best_params = treemodel.param_tuning("lgb")
lgb_best_params

Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds


[I 2023-08-15 09:42:40,860] A new study created in memory with name: no-name-52b52be5-b0a1-4eeb-bdb7-916926be6966


Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds


[I 2023-08-15 09:42:42,414] Trial 0 finished with value: 0.22179991403414676 and parameters: {'num_leaves': 83, 'learning_rate': 0.2543626044014629, 'feature_fraction': 0.5829999619487558, 'bagging_fraction': 0.6164984984262156, 'min_child_samples': 9, 'lambda_l2': 0.05682042146547994, 'lambda_l1': 0.09621377789384544}. Best is trial 0 with value: 0.22179991403414676.


Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839


...

[I 2023-08-15 09:42:47,189] Trial 3 finished with value: 0.22179991403414676 and parameters: {'num_leaves': 89, 'learning_rate': 0.44363063196828606, 'feature_fraction': 0.8181733448025739, 'bagging_fraction': 0.5550223859031387, 'min_child_samples': 8, 'lambda_l2': 0.05358044024355599, 'lambda_l1': 0.06602030091738728}. Best is trial 0 with value: 0.22179991403414676.


Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds


[I 2023-08-15 09:42:48,759] Trial 4 finished with value: 0.22179991403414676 and parameters: {'num_leaves': 46, 'learning_rate': 0.4993053227442533, 'feature_fraction': 0.5980069486480348, 'bagging_fraction': 0.7093654720849973, 'min_child_samples': 3, 'lambda_l2': 0.00519708843347545, 'lambda_l1': 0.0007976489732777315}. Best is trial 0 with value: 0.22179991403414676.


Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839
Value:  0.22179991403414676


{'num_leaves': 83,
 'learning_rate': 0.2543626044014629,
 'feature_fraction': 0.5829999619487558,
 'bagging_fraction': 0.6164984984262156,
 'min_child_samples': 9,
 'lambda_l2': 0.05682042146547994,
 'lambda_l1': 0.09621377789384544}

In [6]:
_, xgb_val_output, xgb_imp_df = treemodel.cv_execute("xgb")
xgb_best_params = treemodel.param_tuning("xgb")
xgb_best_params

[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[294]	eval-logloss:0.07731
[0]	eval-logloss:0.52655
[79]	eval-logloss:0.15150
[0]	eval-logloss:0.50637
[74]	eval-logloss:0.16255
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[145]	eval-logloss:0.15881
[0]	eval-logloss:0.52212
[73]	eval-logloss:0.20678


[I 2023-08-15 09:42:51,139] A new study created in memory with name: no-name-99be5ee6-888f-44f5-b110-9b60e5bb9f88


[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[294]	eval-logloss:0.07731
[0]	eval-logloss:0.52655
[79]	eval-logloss:0.15150
[0]	eval-logloss:0.50637
[74]	eval-logloss:0.16255
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[145]	eval-logloss:0.15881
[0]	eval-logloss:0.52212
[72]	eval-logloss:0.20594


[I 2023-08-15 09:42:53,273] Trial 0 finished with value: 0.31216516835643554 and parameters: {'gamma': 0.7106977884658066, 'max_depth': 3, 'num_leaves': 98, 'learning_rate': 0.12175443843182401, 'min_child_weight': 3, 'max_delta_step': 8, 'subsample': 0.851704793573662, 'lambda': 0.3907230756450786, 'alpha': 0.1996513816127371}. Best is trial 0 with value: 0.31216516835643554.


[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[293]	eval-logloss:0.07732
[0]	eval-logloss:0.52655
[78]	eval-logloss:0.15152
[0]	eval-logloss:0.50637
[74]	eval-logloss:0.16255
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[145]	eval-logloss:0.15881
[0]	eval-logloss:0.52212
[73]	eval-logloss:0.20678


...

[I 2023-08-15 09:42:59,750] Trial 3 finished with value: 0.31216516835643554 and parameters: {'gamma': 0.3859142753268944, 'max_depth': 9, 'num_leaves': 60, 'learning_rate': 0.08431713685031113, 'min_child_weight': 6, 'max_delta_step': 0, 'subsample': 0.8488484897095271, 'lambda': 0.7478864851513223, 'alpha': 0.6644773902479485}. Best is trial 0 with value: 0.31216516835643554.


[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[293]	eval-logloss:0.07732
[0]	eval-logloss:0.52655
[78]	eval-logloss:0.15152
[0]	eval-logloss:0.50637
[74]	eval-logloss:0.16255
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[145]	eval-logloss:0.15881
[0]	eval-logloss:0.52212
[72]	eval-logloss:0.20594


[I 2023-08-15 09:43:01,925] Trial 4 finished with value: 0.31216516835643554 and parameters: {'gamma': 0.023775990018820248, 'max_depth': 5, 'num_leaves': 47, 'learning_rate': 0.41685396975409056, 'min_child_weight': 10, 'max_delta_step': 9, 'subsample': 0.7856050280294459, 'lambda': 0.2035438091566707, 'alpha': 0.3540695808756149}. Best is trial 0 with value: 0.31216516835643554.


Value:  0.31216516835643554


{'gamma': 0.7106977884658066,
 'max_depth': 3,
 'num_leaves': 98,
 'learning_rate': 0.12175443843182401,
 'min_child_weight': 3,
 'max_delta_step': 8,
 'subsample': 0.851704793573662,
 'lambda': 0.3907230756450786,
 'alpha': 0.1996513816127371}

In [7]:
_, cb_val_output, cb_imp_df = treemodel.cv_execute("cb")
cb_best_params = treemodel.param_tuning("cb")
cb_best_params

Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 70.1ms	remaining: 1m 10s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 881ms	remaining: 7.84s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.67s	remaining: 6.65s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.47s	remaining: 5.73s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.26s	remaining: 4.86s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 4.04s	remaining: 4.02s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 4.83s	remaining: 3.2s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 5.62s	remaining: 2.4s
800:	learn: 0.0042617	test: 0.0891180	best: 0.0891180 (800)	total: 6.41s	remaining: 1.59s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.08903141659
bestIteration = 803

Shrink model to first 804 iterations.
Learning rate set to 0.02

[I 2023-08-15 09:43:22,916] A new study created in memory with name: no-name-27cfb330-479c-4ec7-95fa-b8067478391d


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.92s	remaining: 6.78s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 8.28ms	remaining: 8.27s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 805ms	remaining: 7.16s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.59s	remaining: 6.31s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.37s	remaining: 5.51s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.16s	remaining: 4.72s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 3.95s	remaining: 3.93s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 4.73s	remaining: 3.14s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 5.51s	remaining: 2.35s
800:	learn: 0.0042617	tes

[I 2023-08-15 09:43:43,183] Trial 0 finished with value: 0.3645940857907282 and parameters: {'max_depth': 6, 'reg_lambda': 0.474704821152228, 'learning_rate': 0.0911801058783933, 'min_child_samples': 1}. Best is trial 0 with value: 0.3645940857907282.


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.43s	remaining: 5.65s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 8.14ms	remaining: 8.13s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 802ms	remaining: 7.14s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.6s	remaining: 6.35s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.39s	remaining: 5.54s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.17s	remaining: 4.74s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 3.96s	remaining: 3.95s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 4.75s	remaining: 3.15s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 5.54s	remaining: 2.36s
800:	learn: 0.0042617	test

...

[I 2023-08-15 09:44:44,910] Trial 3 finished with value: 0.3645940857907282 and parameters: {'max_depth': 8, 'reg_lambda': 0.9298130305654169, 'learning_rate': 0.4837054201622596, 'min_child_samples': 10}. Best is trial 0 with value: 0.3645940857907282.


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.37s	remaining: 5.51s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 8.04ms	remaining: 8.04s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 806ms	remaining: 7.18s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.6s	remaining: 6.35s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.39s	remaining: 5.56s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.18s	remaining: 4.75s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 3.97s	remaining: 3.96s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 4.75s	remaining: 3.15s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 5.58s	remaining: 2.38s
800:	learn: 0.0042617	test

[I 2023-08-15 09:45:05,657] Trial 4 finished with value: 0.3645940857907282 and parameters: {'max_depth': 7, 'reg_lambda': 0.9427122919899522, 'learning_rate': 0.35377041396447095, 'min_child_samples': 8}. Best is trial 0 with value: 0.3645940857907282.


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.4s	remaining: 5.58s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
Value:  0.3645940857907282


{'max_depth': 6,
 'reg_lambda': 0.474704821152228,
 'learning_rate': 0.0911801058783933,
 'min_child_samples': 1}